# a) Análise Exploratória dos Dados

In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Carregar dados
df = pd.read_csv('enape_raw_dataset.csv')

# Exibir as primeiras linhas
print(df.head())

# Verificar estatísticas básicas
print(df.describe())

# Verificar valores ausentes
print(df.isnull().sum())

# Distribuição da variável PC3_6 (indicador de evasão)
sns.countplot(data=df, x='PC3_6')
plt.title('Distribuição de Evasão (PC3_6)')
plt.xlabel('Evadido (1 = Sim, 0 = Não)')
plt.ylabel('Contagem')
plt.show()


# b) Preparação e Engenharia de Atributos

In [ ]:

# Definir variável alvo
df = df[df['PC3_6'].isin([0, 1])]  # Remove entradas inválidas
df['evadiu'] = df['PC3_6']

# Preencher valores ausentes numéricos com a média
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col] = df[col].fillna(df[col].mean())

# Codificar variáveis categóricas
df = pd.get_dummies(df, drop_first=True)

# Separar features e target
X = df.drop(['evadiu'], axis=1)
y = df['evadiu']

# Normalização
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)


# c) Implementação do Modelo (PyTorch)

In [ ]:

import torch
import torch.nn as nn

class RegressaoLogistica(nn.Module):
    def __init__(self, input_dim):
        super(RegressaoLogistica, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))


# d) Treinamento do Modelo

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Converter para tensores do PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values.reshape(-1, 1), dtype=torch.float32)

# Modelo, loss e otimizador
model = RegressaoLogistica(X.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Treinamento
for epoch in range(100):
    model.train()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Época [{epoch}/100], Loss: {loss.item():.4f}')


# e) Avaliação do Modelo

In [ ]:

# Avaliação
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_class = (y_pred >= 0.5).int()
    
from sklearn.metrics import classification_report, confusion_matrix

print("Matriz de Confusão:")
print(confusion_matrix(y_test_tensor, y_pred_class))
print("
Relatório de Classificação:")
print(classification_report(y_test_tensor, y_pred_class))


# f) Documentação dos Resultados

In [ ]:

# Gráfico de barras para acurácia por classe
import numpy as np

report = classification_report(y_test_tensor, y_pred_class, output_dict=True)
classes = ['Não Evadiu', 'Evadiu']
scores = [report['0']['f1-score'], report['1']['f1-score']]

plt.bar(classes, scores, color=['blue', 'red'])
plt.title('F1-score por classe')
plt.ylabel('F1-score')
plt.ylim(0, 1)
plt.show()
